<img src="img/banner.png" alt="Deparatemento de Ingeniería de Sistemas y Computación, Universidad de los Andes">

# Manejo de varias imágenes en el cubo

**Objetivo**

Este notebook es similar al anterior en el sentido que vamos a realizar las mismas operaciones, pero con dos imágenes.

El objetivo es entender y manejar la dimensión del tiempo.

**Precondiciones - Actividad previa**

1. Tener definido el producto para Sentinel 2 - Ver notebook 1
2. Haber realizado las actividades del notebook 3
3. Tener dos imágenes en el cubo - Ver notebook 2 - **Es hora de indexar la segunda imagen**

**Contenido**

1. Importar librerías
2. Especificación del área de estudio
3. Búsqueda y visualización de imágenes en el Open Data Cube
4. Exploración de la estructura de datos de varias imágenes en el cubo
5. Visualización de las bandas individuales

## 1. Importar librerías
___
En esta sección se importan las librerías cuya funcionalidades particulares son requeridas.

**Nota**: Si le sale un error que dice que no reconoce geopandas es porque no ha ejecutado el notebook 3...

In [ ]:
# las funcionalidades del open data cube son accedidas 
# por medio de la librería datacube
import datacube

# Librería usada para operaciones matemáticas
import numpy as np

# Librería usada para visualización de datos
import matplotlib.pyplot as plt

# Desactiva los warnings en el notebook
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

# Librería usada para la carga de polígonos
import geopandas as gpd

# Configuración de Drivers para leer polígonos en formato KML
gpd.io.file.fiona.drvsupport.supported_drivers['KML'] = 'rw'

## 2. Especificación del área de estudio
___
Para usar el cubo de datos y las imágenes que contiene, lo primero que hay que hacer es contarle al cubo cuál es el área en la que se quiere trabajar, expresada como un rectángulo que define sus límites.

En este notebook, esta área de estudio se especifica a partir de un polígono (en formato .kml), del cual se encuentran sus límites y luego se expande.

In [ ]:
# Carga del archivo .kml y ponerlo en el sistema de coordenadas WGS 84 (EPSG 4326)
df_polygon = gpd.read_file("polygon/tota_lake.kml",driver='KML')
df_polygon = df_polygon.to_crs('EPSG:4326')

# Pintar el polígono del área de estudio
fig, ax = plt.subplots(figsize=(10,10))
df_polygon.boundary.plot(ax=ax,color='red')

# Obtención de la geometría del polígono del GeoDataFrame
geometry_predio = df_polygon['geometry'][0]

# Obtención de los límites del cuadrado que enmarca el polígono
minx, miny, maxx, maxy = geometry_predio.bounds

# Aumento del aŕea del cuadrado para "EPSG:4326" (WGS84 - Unidades en grados)
# 0.001 grados que corresponden a 111 metros alrededor del polígono que define el área de estudio
buffer = 0.001

# Variando el valor de esta variable, se puede tener en cuenta los alrededores del área de estudio. 
# Es claro que mientras más grande sea el área de estudio, más grande es el tiempo para procesarla

minx = minx - buffer
miny = miny - buffer
maxx = maxx + buffer
maxy = maxy + buffer

# Definición de los límites de búsqueda
study_area_lat = (miny,maxy)
study_area_lon = (minx,maxx)

print(f'    latitud= {study_area_lat},')
print(f'    longitud={study_area_lon}')

## 3. Búsqueda y visualización de imágenes en el Open Data Cube
___
El siguiente paso es encontrar entre todas las imágenes que tiene el cubo, aquellas que son de interés para el análisis que se quiere desarrollar. 

Es similar a lo desarrollado en el notebook 3, con la diferencia que ahora debemos obtener dos imágenes en la respuesta

In [ ]:
dc = datacube.Datacube(app="MOOC GEO")

dataset = dc.load(
    product="s2_sen2cor_ard_granule_EO3",                   # El sensor - Sentinel 2
    latitude=study_area_lat,                                # Los límites en latitud del área de estudio
    longitude=study_area_lon,                               # Los límites en longitud del área de estudio
    time=('2021-01-01', '2021-02-01'),                      # El período de tiempo de interés
    measurements=["red","blue","green","nir","swir1","swir2","scl"],  # Las bandas que se quieren estudiar
    crs="EPSG:4326",                                        # El sistema de coordenadas de entrada (WGS 84)
    output_crs="EPSG:4326",                                 # El sistema de coordenadas de la respuesta (WGS 84)
    resolution=(-0.00008983111,0.00008971023)               # Precisión de la respuesta, en grados
)

# Dibujar la imagen en color verdadero (`true color`)
rgb = dataset[["red","green","blue"]].to_array(dim='color')
rgb = rgb.transpose(*(rgb.dims[1:]+rgb.dims[:1]))  # make 'color' the last dimension

# Como ya tenemos más de una imagen, debemos usar parámetros adicionales
img = rgb.plot.imshow(col='time',                  # La dimensión que se va a tomar como referencia: el tiempo
                      col_wrap=4,                  # Cuántas imágenes se dibujan por fila
                      add_colorbar=False,          # Quitar la barra de colores
                      vmin=0, vmax=1200,           # Rango de valores visibles
                      size=10, aspect = 1)         # tamaño y proporción de la visualización

# Con la siguiente instrucción, se muestra también el polígono que sirvió para determinar el área de estudio
for axes in img.axes.flat:
    df_polygon.boundary.plot(ax=axes,markersize=20,color='red',marker='o')
plt.show()

# Notamos que esta celda toma bastante tiempo para generar su resultado. Para mejorar el rendimiento se puede 
# quitar los parámetros size y aspect. 


> Ejercicio: Observe las modificaciones en los resultados y el tiempo para obtener la respuesta con las siuientes modificaciones: 1) cambiando el valor de la variable buffer en el paso 2; 2) cambiando el valor del parámetro size en el paso 3; 3) Quitando los paraámetros size y aspect en el paso 3.

Recordemos que también se puede visualizar las imágenes en *falso color*, con el objetivo de resaltar algunas características de la zona de estudio, no visibles al ojo humano.

Una de las más utilizadas es el *falso color* para resaltar la vegetación, que visualiza la banda del nir (infrarojo cercano) en rojo, la banda del rojo en verde y la banda el verde en azul.


Mayor información y otras combinaciones interesantes en:
- https://acolita.com/lista-de-combinaciones-de-bandas-en-sentinel-2a/

In [ ]:
rgb2 = dataset[["nir","red","green"]].to_array(dim='color')
rgb2 = rgb2.transpose(*(rgb2.dims[1:]+rgb2.dims[:1]))  # make 'color' the last dimension
img2 = rgb2.plot.imshow(col = 'time', col_wrap=4, vmin=0, vmax=1200)

## 4. Exploración de la estructura de datos de varias imágenes en el cubo
___
Al igual que en el notebook 3, miremos cómo es la respuesta ahora que tenemos dos imágenes.

In [ ]:
# Para ver la imagen, en su estructura de datos, simplemente se le pide a Python que la escriba
dataset

Lo primero que encontramos es que la dimensión `time` tiena valor 2

Lo siguiente que debemos observar, es que al mirar en detalle (con el ícono del disco) las variables de datos, tenemos como respuesta dos matrices, cada una correspondiendo con la banda de las dos imágenes que hacen parte de nuestro dataset.
Como no siempre se quiere o es útil trabajar con toda la estructura de datos al tiempo, es posible extraer y trabajar con los elementos indivuduales.

Las siguientes celdas muestran cómo hacerlo

### Exploración de las dimensiones

La propiedad `dims` permite visualizar las dimensiones del dataset.

In [ ]:
dataset.dims

### Exploración de las coordenadas

La propiedad `coords`permite visualizar las coordenadas. Las coordenadas pueden ser vistas como las etiquetas de los ejes de un cubo de tres dimensiones. En este caso, las coordenadas son `time`, `longitude`, `latitude`.

Para la correcta interpretación está especificada el sistema de coordenadas 4326 - WGS 84

In [ ]:
dataset.coords

### Exploración de las fechas de las imágenes

Para conocer de forma explícita las fechas en que fueron tomadas las imágenes resultado de la consulta. Recordemos que en este momento solo hay una imagen en el cubo

In [ ]:
dataset.coords['time']

### Exploración de las coordenadas de longitud

Para conocer de forma explícita las coordendas de longitud

In [ ]:
dataset.coords['longitude']

### Exploración de las coordenadas de latitud

Para conocer de forma explícita las coordendas de latitud

In [ ]:
dataset.coords['latitude']

### Exploración de las variables de datos

La información espectral de una imagen satelital está organizada por el Open Data Cube en *variables de datos*. 

Cada variable de datos contiene la información de una única banda. 

La información de cada banda es organizada en un arreglo de tres dimensiones. 

Para acceder a la información espectral de la banda `blue` se puede usar la expresión mostrada a continuación.

In [ ]:
dataset.blue

Dado que cada variable de datos es un arreglo de tres dimensiones, es posible indexar la información del mismo. Es importante conocer el orden de los ejes para determinar de antemano la información que se muestra al indexar una variable de datos.

Por ejemplo, puedo obtener la segunda imagen en el tiempo para la banda `blue`. 

In [ ]:
dataset.blue[1]

De la misma forma, se puede ver el valor para la banda `blue` de un único píxel. Los valores entre corchetes `[1,0,0]` se interpretan considerando el orden de las coordenadas. En este caso:

* la primera coordenada es el tiempo (`time`) de modo que en la matrix se ha seleccionado el periodo de tiempo `1`. 
* la segunda coodenada es la latitud (`latitude`) de modo que en la matrix se ha seleccionado la latitud `0`
* la tercera coordenada es la longitud (`longitude`) así, la longitud seleccionada es `0`

In [ ]:
dataset.blue[1,0,0]

### Exploración de atributos

Entre los atributos que hacen parte de los metadatos de la imagen, se muestra el sistema de referencia de coordenadas (CRS) de la imagen obtenida a partir del Open Data Cube. Una de las bondades del Open Data Cube es que permite obtener la información de las imágenes en diferentes sistemas de coordendas y en diferentes dimensiones.

In [ ]:
dataset.crs

## 5. Visualización de las bandas individuales
___
Para visualizar una banda particular se utiliza el comando `plot` a la variable de datos deseada, utilizando la paleta de colores por defecto.

In [ ]:
dataset.blue.plot(col='time', vmin=0,vmax=1200)

Para cambiar el rango de colores de la visualización mostrada se usa el parámetro `cmap` que indica la paleta de colores que se quiere utilizar.

El listado de paletas de colores válids está [aquí](https://matplotlib.org/stable/tutorials/colors/colormaps.html)

In [ ]:
dataset.blue.plot(col='time', vmin=0,vmax=1200, cmap='Blues')

In [ ]:
dataset.red.plot(col='time', cmap='Reds',vmin=0,vmax=1200)

> **Ejercicio:**  Realice el proceso de **Visualización** para las variables (bandas) restantes `green`,`nir`,`swir1`,`swir2` y `scl`. Cree una nueva celda para cada ejemplo. Puede apoyarse en el ejemplo de exploración de la banda `blue` mostrado anteriormente. Use los colores `'Greens'` para la banda `green`; para las demás bandas no cambie los colores.